In [2]:

import pandas as pd


In [4]:
pd.read_csv("final_results_paws.csv")

,id,premise_,hypothesis_,amr_h,amr_p,ground_truth,pred,pred_amr,label_direct,label_amr,helpfulness,did_llm_failed
0,paws_test_0000,"The NRA has referred to the NAGR leader, Dudle...",The NRA has referred to the NAGR - leader Al S...,(r / refer-01~3\n :ARG0 (o / organization~1...,(r / refer-01~3\n :ARG0 (o / organization~1...,0.0,0,0.0,1,1,0,0
1,paws_test_0001,The layout of Belmont is similar to that of Tw...,The Belmont layout is similar to that of Twin ...,(r / resemble-01~4\n :ARG1 (l / layout~2\n ...,(r / resemble-01~5\n :ARG1 (l / layout~1\n ...,1.0,1,1.0,1,1,0,0
2,paws_test_0002,Road access to Island Harbour is via Mill Lane...,The access to North Fairlee Road is via Mill L...,(a / access-01~1\n :ARG1 (r / road~5\n ...,(a / access-01~1\n :ARG1 (r / road~0)\n ...,0.0,0,0.0,1,1,0,0
3,paws_test_0003,Stoya was born to Jessica Stoyadinovich in Wil...,Stoya was born as Jessica Stoyadinovich in Wil...,(b / bear-02~2\n :ARG0 (a / and~15\n ...,(b / bear-02~2\n :ARG0 (a / and~15\n ...,1.0,1,0.0,1,0,-1,0
4,paws_test_0004,"Having won the 2001 Super League, the Knights ...","After winning the Super League 2001, the knigh...",(t3 / travel-01~9\n :purpose (p / play-01~1...,(t3 / travel-01~9\n :purpose (p / play-01~1...,1.0,1,1.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,paws_test_7995,"Atopic dermatitis is sometimes called eczema, ...","Atopic dermatitis is also called eczema, a ter...",(c / call-01~4\n :ARG1 (d / disease~1\n ...,"(c / call-01~4\n :ARG1 ""dermatitis""~1\n ...",0.0,1,0.0,0,1,1,1
7996,paws_test_7996,The Ascunsu River is a tributary of the River ...,The river Suceava is a tributary of the Ascuns...,(t / tributary~5\n :domain (r / river~1\n ...,(t / tributary~5\n :domain (r / river~2\n ...,0.0,0,1.0,1,0,-1,0
7997,paws_test_7997,"In 1847, Lycoming County was formed from the S...",Sullivan County was formed from Lycoming Count...,(a2 / and~10\n :op1 (f / form-01~3\n ...,(a2 / and~12\n :op1 (f / form-01~6\n ...,0.0,0,0.0,1,1,0,0
7998,paws_test_7998,Kalisch wrote and anticipated important points...,Kalisch anticipated before the publication of ...,(a / and~11\n :op1 (a2 / anticipate-01~1\n ...,(a / and~2\n :op1 (w2 / write-01~1\n ...,0.0,0,0.0,1,1,0,0


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np


# Load your labeled data into a pandas DataFrame
data = pd.read_csv("final_results_paws.csv")


data=data.assign(label=np.where(data.helpfulness<=0,0,1))

columns_to_lower = ['premise_', 'hypothesis_', 'amr_h', 'amr_p']

# Convert specified columns to lowercase
for col in columns_to_lower:
    data[col] = data[col].apply(lambda x: x.lower())


# Splitting the data
train_data, test_data, train_labels, test_labels = train_test_split(
    data[columns_to_lower], data['helpfulness'], test_size=0.2, random_state=42
)


# Feature extraction for the 'amr_text' column
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features
train_features = tfidf_vectorizer.fit_transform(train_data['amr_h'])
test_features = tfidf_vectorizer.transform(test_data['amr_h'])

# Model training
model = LogisticRegression()  # You can try other models too
model.fit(train_features, train_labels)

# Model evaluation
predictions = model.predict(test_features)
report = classification_report(test_labels, predictions)
print(report)


              precision    recall  f1-score   support

          -1       0.50      0.01      0.01       294
           0       0.70      1.00      0.82      1112
           1       0.30      0.02      0.03       194

    accuracy                           0.70      1600
   macro avg       0.50      0.34      0.29      1600
weighted avg       0.61      0.70      0.58      1600



/Users/nils/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
import pandas as pd 
import numpy as np
df=pd.read_csv("final_results_paws.csv")
dataset='PAWS'
outcome_variable='helpfulness'

if dataset in ['PAWS']:
    df=df.assign(text="Sentence 1: "+df.premise_+"\nAMR 1: "+df.amr_p+"\nSentence 2: "+df.hypothesis_+"\nAMR 2: "+df.amr_h)
elif dataset in ['translation','logic','django','spider']:
    df=df.assign(text="Text: "+df.text+"\nAMR: "+df.amr)
elif dataset in ['pubmed']:
    df=df.assign(text="Text: "+df.text+"\nInteraction: "+df.interaction+"\nAMR: "+df.amr)
df=df.assign(label=np.where(df.helpfulness<=0,0,1))
df=df.loc[:,['id','text','label']]
df=df.loc[~df.text.isna()]

In [8]:
df

,id,text,label
0,paws_test_0000,Sentence 1: The NRA has referred to the NAGR l...,0
1,paws_test_0001,Sentence 1: The layout of Belmont is similar t...,0
2,paws_test_0002,Sentence 1: Road access to Island Harbour is v...,0
3,paws_test_0003,Sentence 1: Stoya was born to Jessica Stoyadin...,0
4,paws_test_0004,"Sentence 1: Having won the 2001 Super League, ...",0
...,...,...,...
7995,paws_test_7995,Sentence 1: Atopic dermatitis is sometimes cal...,1
7996,paws_test_7996,Sentence 1: The Ascunsu River is a tributary o...,0
7997,paws_test_7997,"Sentence 1: In 1847, Lycoming County was forme...",0
7998,paws_test_7998,Sentence 1: Kalisch wrote and anticipated impo...,0


In [18]:
df.value_counts()


id              text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [19]:
(df['label'] == 0).sum()

6966

In [21]:
print(df.head())

               id                                               text  label
0  paws_test_0000  Sentence 1: The NRA has referred to the NAGR l...      0
1  paws_test_0001  Sentence 1: The layout of Belmont is similar t...      0
2  paws_test_0002  Sentence 1: Road access to Island Harbour is v...      0
3  paws_test_0003  Sentence 1: Stoya was born to Jessica Stoyadin...      0
4  paws_test_0004  Sentence 1: Having won the 2001 Super League, ...      0


In [22]:
print(df.columns)

Index(['id', 'text', 'label'], dtype='object')


In [19]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
import torch
from transformers import Trainer, TrainingArguments
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import set_seed
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,classification_report
from datasets import load_metric
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from torch import nn
import transformers
import wandb
from transformers import get_linear_schedule_with_warmup
import os

def compute_metrics_discrete(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    cr=classification_report(labels,predictions,output_dict=True)
    recall_w = recall_score(y_true=labels, y_pred=predictions,average='weighted')
    precision_w = precision_score(y_true=labels, y_pred=predictions,average='weighted')
    f1_micro = f1_score(y_true=labels, y_pred=predictions,average='micro')
    f1_weighted = f1_score(y_true=labels, y_pred=predictions,average='weighted')
    return {"accuracy": accuracy, "f1_0":cr['0']['f1-score'],"f1_1":cr['1']['f1-score'],
            "precision_1":cr['1']['precision'],"recall_1":cr['1']['recall'],
             "precision_w": precision_w, "recall_w": recall_w,
            "f1_micro": f1_micro,"f1_weighted": f1_weighted} 

def process_data(file_path,dataset,amr=True,outcome_variable='helpfulness'):
    """Process data for training RoBERTa model, formatting depends on the dataset"""
    df=pd.read_csv(file_path)
    if amr:
        if dataset in ['PAWS']:
            df=df.assign(text="Sentence 1: "+df.premise_+"\nAMR 1: "+df.amr_p+"\nSentence 2: "+df.hypothesis_+"\nAMR 2: "+df.amr_h)
        elif dataset in ['translation','logic','django','spider']:
            df=df.assign(text="Text: "+df.text+"\nAMR: "+df.amr)
        elif dataset in ['pubmed']:
            df=df.assign(text="Text: "+df.text+"\nInteraction: "+df.interaction+"\nAMR: "+df.amr)
    else:
        if dataset in ['PAWS']:
            df=df.assign(text="Sentence 1: "+df.amr_p+"\nSentence 2: "+df.hypothesis_)
        elif dataset in ['translation','logic','django','spider']:
            df=df.assign(text="Text: "+df.text)
        elif dataset in ['pubmed']:
            df=df.assign(text="Text: "+df.text+"\nInteraction: "+df.interaction)
    
    if outcome_variable=='helpfulness':
        df=df.assign(label=np.where(df.helpfulness<=0,0,1))
    elif outcome_variable=='did_llm_failed':
        df=df.assign(label=df.did_llm_failed)
    df=df.loc[:,['id','text','label']]
    df=df.loc[~df.text.isna()]
    return df

def split_sets(dataset,df):
    """Split data into train, dev and test sets, formatting depends on the dataset"""
    if dataset in ['translation']:
        df['set']=df.id.str[:10]
        train_set=df.loc[df['set']=='newstest13']
        dev_set, test_set = train_test_split(df.loc[df['set']=='newstest16'], test_size=0.5,random_state=42)
    elif dataset in ['PAWS','pubmed']:
        train_set, val_df = train_test_split(df, test_size=0.3,random_state=42)
        dev_set, test_set = train_test_split(val_df, test_size=0.5,random_state=42)
    elif dataset in ['logic','django','spider']:
        train_set=df.loc[df['id'].str.contains('train')]
        test_set=df.loc[df['id'].str.contains('test')]
        dev_set=df.loc[df['id'].str.contains('dev')]
    
    return train_set,dev_set,test_set


def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)



def model_init():
    transformers.set_seed(42)
    m = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=2,device_map='auto')
    m.classifier.dropout.p = 0.3  # Adjust the dropout probability as needed
    m.roberta.apply(freeze_weights)
    for name, param in m.classifier.named_parameters():
        param.requires_grad = True
    return m

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 2 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=model.device,dtype=torch.float))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def freeze_weights(m):
    for name, param in m.named_parameters():
        param.requires_grad = False  


sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'eval/loss',
    'goal': 'minimize'   
    }
"""
parameters_dict = {
    "learning_rate": {"distribution": "uniform", "min": 2e-5, "max": 1e-4},
    "per_device_train_batch_size": {"values": [16,32 ]},
    }
"""
parameters_dict = {
    "learning_rate": {"values": [2e-5]},
    "per_device_train_batch_size": {"values": [8 ]},
    }
sweep_config['parameters'] = parameters_dict

sweep_config['metric'] = metric
#sweep_id = wandb.sweep(sweep_config, project="helpfulness")


dataset='django'
#datasets=['PAWS','translation','pubmed','logic','django','spider']
## True for balancing the observations in the loss function (currently not working)
compute_weights=False
current=-1
d_metric='f1_1'
amr_flag=True
decision_metric='eval_'+d_metric
outcome_variable='helpfulness'
## final results files
##https://drive.google.com/drive/folders/17pwdiiu7U1oyly8YwMtqCRdu3GBIWT3K
file_path='../final_results_django_corrected.csv'
logs_path=''
run_name=dataset+"_hyp_final_"+outcome_variable

df=process_data(file_path=file_path,dataset=dataset,amr=amr_flag,outcome_variable=outcome_variable)
train_set,dev_set,test_set=split_sets(dataset=dataset,df=df)


if compute_weights:
    class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=train_set.label.unique(), y=train_set.label.values)
else:
    class_0_count = len(train_set[train_set["label"] == 0])
    class_1_count = len(train_set[train_set["label"] == 1])
    total = class_0_count + class_1_count
    class_weights = [total/class_0_count, total/class_1_count]


## prepare sets
set_seed(42)
torch.manual_seed(42)
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

train_dataset=Dataset.from_pandas(train_set)
val_dataset=Dataset.from_pandas(dev_set)
test_dataset=Dataset.from_pandas(test_set)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|██████████| 1603/1603 [00:00<00:00, 2623.82 examples/s]


In [20]:
train_set

,id,text,label
2433,django_train_0000,Text: define the method i with an argument sel...,0
2434,django_train_0001,Text: if out has an 'isatty' argument and retu...,0
2435,django_train_0002,Text: raise an ValueError exception with an ar...,0
2436,django_train_0003,Text: return is empty dictionary.\nAMR: (d / d...,0
2437,django_train_0004,Text: get the value under the key 'database' o...,0
...,...,...,...
14336,django_train_11903,Text: call the method time.mktime with an argu...,0
14337,django_train_11904,Text: define the method u with an argument sel...,0
14338,django_train_11905,Text: substitute size for self.size.\nAMR: (s3...,0
14339,django_train_11906,Text: self.keyOrder is an empty list.\nAMR: (m...,1


In [21]:
print((train_set['label'] == 1).sum())
print((train_set['label'] == 0).sum())

2236
9672


2537